In [1]:
import requests
import pandas as pd
import time
import numpy as np

In [2]:
def fetch_live_data():
    url = 'https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT'
    response = requests.get(url)
    data = response.json()
    return float(data['price'])

def fetch_historical_data(symbol, interval, limit=1000):
    """
    interval:
    1m, 3m, 5m, 15m, 30m
    1h, 2h, 4h, 6h, 8h, 12h,
    1d, 3d, 1w, 1M
    """
    url = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit={limit}'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
    return df
    
def preprocess_data(df):
    df['Returns'] = df['Close'].pct_change()
    # df.dropna(inplace=True)
    return df

# Function to update the DataFrame with new live data
def update_data(df):
    price = fetch_live_data()
    timestamp = pd.Timestamp.now()
    new_data = pd.DataFrame({'Date': [timestamp], 'Close': [price]})
    df = pd.concat([df, new_data])
    df.set_index('Date', inplace=True)
    df = preprocess_data(df)
    return df

In [ ]:
from agent_ma import MovingAverageCrossoverAgent
from agent_dummy import DummyAgent

# Initialize DataFrame to store live data
df = pd.DataFrame(columns=['Date', 'Close'])

# Initialize agents
mac_agent = MovingAverageCrossoverAgent()
dummy_agent = DummyAgent()

# Function to update agents and print portfolio values
def update_agents(df, t):
    df = update_data(df)
    current_price = df['Close'].iloc[-1]
    mac_agent.trade(df)
    dummy_agent.trade(df)
    print(f"MAC Agent Portfolio Value: {mac_agent.get_portfolio_value(current_price)}")
    print(f"Dummy Agent Portfolio Value: {dummy_agent.get_portfolio_value(current_price)}")

    return df

interval = 1
t = 0
while True:
    df = update_agents(df, t)
    time.sleep(interval)  # Fetch data and trade every minute
    t += 1

In [7]:
def fetch_historical_data(symbol, interval, limit=1000):
    url = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit={limit}'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
    return df


fetch_historical_data("BTCUSDT", "1m")

,open,high,low,close,volume
timestamp,,,,,
2024-07-04 04:30:00,58804.23,58816.00,58736.00,58765.80,25.13337
2024-07-04 04:31:00,58765.80,58841.71,58765.79,58841.71,20.91068
2024-07-04 04:32:00,58841.70,58868.00,58840.06,58867.99,10.55487
2024-07-04 04:33:00,58868.00,58883.96,58868.00,58883.96,4.21190
2024-07-04 04:34:00,58883.96,58947.87,58883.95,58947.86,9.30069
...,...,...,...,...,...
2024-07-04 21:05:00,58333.90,58333.90,58314.22,58316.54,12.85240
2024-07-04 21:06:00,58316.54,58316.54,58288.33,58288.33,8.22819
2024-07-04 21:07:00,58288.33,58327.78,58288.33,58327.78,4.93723
